In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

def search_restaurant(restaurant_name):
    # 웹드라이버 설정
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    
    # 네이버 지도 페이지 열기
    driver.get("https://map.naver.com/p?c=15.00,0,0,0,dh")
    
    try:
        # 검색 입력 필드가 로드될 때까지 대기
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input.input_search"))
        )
        
        # 검색어 입력
        search_box.send_keys(restaurant_name)
        search_box.send_keys(Keys.RETURN)
        
        # 검색 결과가 로드될 때까지 대기 (여기서는 간단히 3초 대기)
        time.sleep(3)
        
        # 여기에 검색 결과를 처리하는 코드를 추가할 수 있습니다.
        # 예: 검색 결과 목록 가져오기, 특정 정보 추출 등
        
    except Exception as e:
        print(f"오류 발생: {e}")
    
    finally:
        # 브라우저 종료
        driver.quit()

# 사용 예
search_restaurant("맛불")

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

def setup_driver():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # 브라우저를 표시하지 않고 실행
    return webdriver.Chrome(service=service, options=options)

def extract_reviews(driver, url, num_reviews=10):
    driver.get(url)
    reviews = []
    
    try:
        # 리뷰 섹션으로 이동
        review_tab = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='review']"))
        )
        review_tab.click()
        
        # 리뷰 로딩 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "li.pui__X35jYm"))
        )
        
        while len(reviews) < num_reviews:
            review_elements = driver.find_elements(By.CSS_SELECTOR, "li.pui__X35jYm")
            
            for element in review_elements:
                if len(reviews) >= num_reviews:
                    break
                
                username = element.find_element(By.CSS_SELECTOR, "span.pui__NMi-Dp").text
                review_text = element.find_element(By.CSS_SELECTOR, "a.pui__xtsQN-").text
                
                review_stats = element.find_elements(By.CSS_SELECTOR, "span.pui__WN-kAf")
                review_count = review_stats[0].text if len(review_stats) > 0 else ''
                photo_count = review_stats[1].text if len(review_stats) > 1 else ''
                
                keywords = [span.text for span in element.find_elements(By.CSS_SELECTOR, "span.pui__V8F9nN")]
                keywords_str = ', '.join(keywords)
                
                visit_info = element.find_element(By.CSS_SELECTOR, "div.pui__QKE5Pr")
                visit_date = visit_info.find_element(By.TAG_NAME, "time").text
                visit_count = visit_info.find_elements(By.CSS_SELECTOR, "span.pui__gfuUIT")[1].text
                
                reviews.append({
                    'Username': username,
                    'Review Text': review_text,
                    'Review Count': review_count,
                    'Photo Count': photo_count,
                    'Keywords': keywords_str,
                    'Visit Date': visit_date,
                    'Visit Count': visit_count
                })
            
            # 다음 페이지로 이동
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "a.pui__HK56z")
                next_button.click()
                time.sleep(2)  # 페이지 로딩 대기
            except:
                break  # 다음 페이지가 없으면 종료
    
    except Exception as e:
        print(f"An error occurred: {e}")
    
    return reviews

def save_to_csv(data, filename='naver_map_reviews.csv'):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Data saved to {filename}")

# 메인 실행 코드

url = "https://map.naver.com/p/smart-around/place/1647171960?c=15.00,0,0,0,dh&placePath=/review"  # 실제 장소 URL로 변경해야 함
driver = setup_driver()
reviews = extract_reviews(driver, url, num_reviews=50)  # 원하는 리뷰 수 지정
driver.quit()
save_to_csv(reviews)

An error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00477143+25587]
	(No symbol) [0x0040A2E4]
	(No symbol) [0x00302113]
	(No symbol) [0x00346F62]
	(No symbol) [0x003471AB]
	(No symbol) [0x00387852]
	(No symbol) [0x0036ABE4]
	(No symbol) [0x00385370]
	(No symbol) [0x0036A936]
	(No symbol) [0x0033BA73]
	(No symbol) [0x0033C4CD]
	GetHandleVerifier [0x00754C63+3030803]
	GetHandleVerifier [0x007A6B99+3366473]
	GetHandleVerifier [0x005095F2+624802]
	GetHandleVerifier [0x00510E6C+655644]
	(No symbol) [0x00412C9D]
	(No symbol) [0x0040FD68]
	(No symbol) [0x0040FF05]
	(No symbol) [0x00402336]
	BaseThreadInitThunk [0x76B0FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x771780CE+286]
	RtlGetAppContainerNamedObjectPath [0x7717809E+238]

Data saved to naver_map_reviews.csv


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException,  ElementClickInterceptedException
import pandas as pd
import time

def setup_driver():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    return webdriver.Chrome(service=service, options=options)

def wait_and_find_element(driver, by, value, timeout=10):
    try:
        element = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((by, value))
        )
        return element
    except TimeoutException:
        print(f"Element not found: {value}")
        return None
    
def cut_address(addr):
    parts = addr.split()
    if len(parts) >= 3:
        return ' '.join(parts[:3])
    else:
        return addr  # 3개 미만의 부분이 있는 경우 원래 주소를 반환



# 'cp949' 인코딩을 사용하여 CSV 파일을 다시 로드해봅니다.
data_pf = pd.read_csv('식당정보.csv', encoding='cp949')

data_pf.head()



,MCT_YM,UE_CNT_GRP,UE_AMT_GRP,UE_AMT_PER_TRSN_GRP,MON_UE_CNT_RAT,TUE_UE_CNT_RAT,WED_UE_CNT_RAT,THU_UE_CNT_RAT,FRI_UE_CNT_RAT,SAT_UE_CNT_RAT,...,RC_M12_AGE_30_CUS_CNT_RAT,RC_M12_AGE_40_CUS_CNT_RAT,RC_M12_AGE_50_CUS_CNT_RAT,OP_YMD,MCT_TYPE,ADDR,MCT_NM,예상영업시간,휴업일,ADDR2
0,(유)아웃백스테이크하우스 제주아일랜드점_20090217,5.833333,6.000000,5.0,0.119689,0.116009,0.114095,0.099294,0.139473,0.209187,...,0.339750,0.356750,0.115250,20090217,패밀리 레스토랑,제주 제주시 오라이동 3173번지 1층,아웃백스테이크하우스 제주아일랜드점,12~23,없음,제주 제주시 오라이동
1,(주) 베이힐_20140110,1.500000,5.833333,6.0,0.182525,0.102288,0.148416,0.081039,0.129945,0.133749,...,0.343750,0.320833,0.158000,20140110,단품요리 전문,제주 서귀포시 하예동 1888번지,베이힐,5~23,없음,제주 서귀포시 하예동
2,(주) 신세계푸드 제주신화월드 고래라면_20171227,5.083333,4.166667,2.0,0.137782,0.131685,0.121358,0.130520,0.151154,0.162207,...,0.333917,0.494417,0.085417,20171227,구내식당/푸드코트,제주 서귀포시 안덕면 서광리 2489번지 1층,신세계푸드 제주신화월드 고래라면,12~23,없음,제주 서귀포시 안덕면
3,(주) 신세계푸드 제주신화월드 윤경양식당_20171228,5.750000,5.416667,2.0,0.133577,0.126770,0.113996,0.133658,0.139016,0.165438,...,0.374083,0.481083,0.068667,20171228,구내식당/푸드코트,제주 서귀포시 안덕면 서광리 2489번지 1층,신세계푸드 제주신화월드 윤경양식당,12~23,없음,제주 서귀포시 안덕면
4,(주) 신세계푸드 제주신화월드 티플레이스_20171227,5.416667,6.000000,4.0,0.128997,0.118792,0.122868,0.129584,0.138668,0.176090,...,0.302000,0.473750,0.119333,20171227,구내식당/푸드코트,제주 서귀포시 안덕면 서광리 2489번지 1층,신세계푸드 제주신화월드 티플레이스,12~23,없음,제주 서귀포시 안덕면


In [2]:
# 새로운 'ADDR_filtered' 열 생성
data_pf['ADDR_filtered'] = data_pf['ADDR'].apply(cut_address)

for li in ["주소",'영업시간', '전화번호', '메뉴', 'error']:
    data_pf[li]=""

driver = setup_driver()
for i in range(len(data_pf)):
    restaurant_name = data_pf['MCT_NM'][i]
    target_address = data_pf['ADDR_filtered'][i]
    try:
        # 네이버 지도 페이지 열기
        driver.get("https://map.naver.com/p?c=15.00,0,0,0,dh")
        current_url=""
        # 검색 입력 필드가 로드될 때까지 대기
        search_box = wait_and_find_element(driver, By.CSS_SELECTOR, "input.input_search")
        if not search_box:
            print("Search box not found")
        
        # 검색어 입력
        search_box.send_keys(restaurant_name)
        search_box.send_keys(Keys.RETURN)

        time.sleep(3)  # 검색 결과 로딩 대기

        #iframe으로 전환
        iframe = wait_and_find_element(driver, By.ID, "searchIframe")
        if not iframe:
            print("Entry iframe not found")


        driver.switch_to.frame(iframe)

        restaurant_elements = driver.find_elements(By.CSS_SELECTOR, "li.VLTHu.OW9LQ")

        target_restaurant = None
        for element in restaurant_elements:
            address_element = element.find_element(By.CSS_SELECTOR, "span.Pb4bU")
            if target_address in address_element.text:
                target_restaurant = element
        print("dddd")
        # 찾은 식당 클릭
        restaurant_link = target_restaurant.find_element(By.CSS_SELECTOR, "a.P7gyV")
        restaurant_link.click()
        time.sleep(3)  # 페이지 로딩 대기
        
        iframe = wait_and_find_element(driver, By.ID, "entryIframe")
        if not iframe:
            print("Entry iframe not found")
        driver.switch_to.frame(iframe)
        print("dddd1")

        # 영업시간 추출
        time_element = wait_and_find_element(driver, By.CSS_SELECTOR, ".nbXkr")
        if time_element:
            time_info = time_element.text
            data_pf.loc[data_pf.index[i], '영업시간'] = time_info
            print(time_info)



        # 전화번호 추출
        phone_element = wait_and_find_element(driver, By.CSS_SELECTOR, ".xlx7Q")
        if phone_element:
            phone_number = phone_element.text
            data_pf.loc[data_pf.index[i], '전화번호'] = phone_number
            print(phone_number)

        # 메뉴 리스트 추출
        menu_items = driver.find_elements(By.CSS_SELECTOR, "li.E2jtL")
        menu_list = []
        print(menu_list)
        for item in menu_items:
            menu_name = item.find_element(By.CSS_SELECTOR, ".lPzHi").text
            price_element = item.find_element(By.CSS_SELECTOR, ".GXS1X")
            price = price_element.text.replace("원", "").replace(",", "").strip()
            if price == "무료":
                price = "0"
            menu_list.append(f"{menu_name}:{price}")

        print("dddd2")
        menu_string = ", ".join(menu_list)
        data_pf.loc[data_pf.index[i], '메뉴'] = menu_string
        time.sleep(3)  # 페이지 로딩 대기

        data_pf.loc[data_pf.index[i], 'error'] = ""
    except Exception as e:
        #print(f"An error occurred: {e}")
        data_pf.loc[data_pf.index[i], 'error'] = e
    finally:
        # 현재 URL 가져오기
        current_url = driver.current_url
        data_pf.loc[data_pf.index[i], '주소'] = current_url
        print(current_url)
        # 기본 프레임으로 돌아가기
        driver.switch_to.default_content()

data_pf.to_csv('식당정보_naver.csv', index=False, encoding='cp949')

# 테마 선택 임베딩
# 유저가 입력한 결과에 대해 

dddd
https://map.naver.com/p/search/%EC%95%84%EC%9B%83%EB%B0%B1%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%ED%95%98%EC%9A%B0%EC%8A%A4%20%EC%A0%9C%EC%A3%BC%EC%95%84%EC%9D%BC%EB%9E%9C%EB%93%9C%EC%A0%90/place/1418972938?c=15.00,0,0,0,dh&isCorrectAnswer=true
dddd
Element not found: entryIframe
Entry iframe not found
dddd1
Element not found: .nbXkr
Element not found: .xlx7Q
[]
dddd2
https://map.naver.com/p/search/%EB%B2%A0%EC%9D%B4%ED%9E%90/place/1463424775?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp
dddd
https://map.naver.com/p/search/%EC%8B%A0%EC%84%B8%EA%B3%84%ED%91%B8%EB%93%9C%20%EC%A0%9C%EC%A3%BC%EC%8B%A0%ED%99%94%EC%9B%94%EB%93%9C%20%20%EA%B3%A0%EB%9E%98%EB%9D%BC%EB%A9%B4?c=15.00,0,0,0,dh
dddd
https://map.naver.com/p/search/%EC%8B%A0%EC%84%B8%EA%B3%84%ED%91%B8%EB%93%9C%20%EC%A0%9C%EC%A3%BC%EC%8B%A0%ED%99%94%EC%9B%94%EB%93%9C%20%20%EC%9C%A4%EA%B2%BD%EC%96%91%EC%8B%9D%EB%8B%B9?c=15.00,0,0,0,dh
dddd
https://map.naver.com/p/search/%EC%8B%A0%EC%84%B8%EA%B3%84%ED%91%B8%EB%93%9C%20%EC%A0%9C%EC%A3%B

KeyboardInterrupt: 

In [3]:
data_pf.to_csv('식당정보_naver1.csv', index=False, encoding='cp949')

len(data_pf)

2976

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time

def setup_driver():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    return webdriver.Chrome(service=service, options=options)

def wait_and_find_element(driver, by, value, timeout=10):
    try:
        element = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((by, value))
        )
        return element
    except TimeoutException:
        print(f"Element not found: {value}")
        return None

def search_and_extract_reviews(restaurant_name):
    driver = setup_driver()
    reviews = []
    
    try:
        # 네이버 지도 페이지 열기
        driver.get("https://map.naver.com/p?c=15.00,0,0,0,dh")
        
        # 검색 입력 필드가 로드될 때까지 대기
        search_box = wait_and_find_element(driver, By.CSS_SELECTOR, "input.input_search")
        if not search_box:
            print("Search box not found")
            return reviews
        
        # 검색어 입력
        search_box.send_keys(restaurant_name)
        search_box.send_keys(Keys.RETURN)

        time.sleep(3)  # 검색 결과 로딩 대기

        # # iframe으로 전환
        # iframe = wait_and_find_element(driver, By.ID, "searchIframe")
        # if not iframe:
        #     print("Entry iframe not found")
        #     return reviews
        
        # driver.switch_to.frame(iframe)
        
        # 검색 결과에서 정확한 식당 선택
        restaurant_link = wait_and_find_element(driver, By.CSS_SELECTOR, "a.P7gyV")
        if not restaurant_link:
            print("Restaurant link not found")
            return reviews
        
        restaurant_link.click()

        # 현재 URL 가져오기
        current_url = driver.current_url
        return current_url

    except Exception as e:
        print(f"An error occurred: {e}")
    
    finally:
        driver.quit()
    
    return reviews


restaurant_name = "맛불"  # 검색할 식당 이름
reviews = search_and_extract_reviews(restaurant_name)  # 원하는 리뷰 수 지정
print(reviews)

https://map.naver.com/p/search/%EB%A7%9B%EB%B6%88/place/1647171960?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp


In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import csv
import time
import re

def extract_number(text):
    return re.search(r'\d+', text).group() if re.search(r'\d+', text) else ''

def scrape_reviews(url):
    driver = webdriver.Chrome()
    driver.get(url)

    reviews = []
    
    try:
        # 리뷰 더 보기 버튼이 있는 경우 클릭
        while True:
            try:
                more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.fvwqf"))
                )
                more_button.click()
                time.sleep(1)
            except TimeoutException:
                break

        # 리뷰 요소들 찾기
        review_elements = driver.find_elements(By.CSS_SELECTOR, "li.pui__X35jYm")

        # 리뷰 정보 추출
        for element in review_elements:
            try:
                author = element.find_element(By.CSS_SELECTOR, "span.pui__NMi-Dp").text
                content = element.find_element(By.CSS_SELECTOR, "a.pui__xtsQN-").text
                visit_date = element.find_element(By.CSS_SELECTOR, "time").text
                visit_count = element.find_element(By.CSS_SELECTOR, "span.pui__gfuUIT:nth-of-type(2)").text
                
                # 리뷰 수와 사진 수 추출
                stats = element.find_elements(By.CSS_SELECTOR, "span.pui__WN-kAf")
                review_count = extract_number(stats[0].text) if len(stats) > 0 else ''
                photo_count = extract_number(stats[1].text) if len(stats) > 1 else ''
                
                reviews.append({
                    "author": author,
                    "content": content,
                    "visit_date": visit_date,
                    "visit_count": visit_count,
                    "review_count": review_count,
                    "photo_count": photo_count
                })
            except NoSuchElementException:
                continue  # 일부 정보가 없는 경우 해당 리뷰 건너뛰기

    finally:
        driver.quit()

    return reviews

def save_to_csv(reviews, filename="reviews.csv"):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["author", "content", "visit_date", "visit_count", "review_count", "photo_count"])
        writer.writeheader()
        for review in reviews:
            writer.writerow(review)


url = "https://pcmap.place.naver.com/restaurant/1647171960/review/visitor"
reviews = scrape_reviews(url)
save_to_csv(reviews)
print(f"{len(reviews)} 개의 리뷰가 추출되어 reviews.csv 파일에 저장되었습니다.")

47 개의 리뷰가 추출되어 reviews.csv 파일에 저장되었습니다.
